[![Binder](https://mybinder.org/badge_logo.svg)](https://lab.mlpack.org/v2/gh/mlpack/examples/master?urlpath=lab%2Ftree%2Freinforcement_learning_gym%2Fcartpole_dqn%2Fcartpole_dqn.ipynb)

You can easily run this notebook at https://lab.mlpack.org/

This notebook shows how to get started with training reinforcement learning agents, particularly DQN agents, using mlpack. Here, we train a [Simple DQN](https://www.cs.toronto.edu/~vmnih/docs/dqn.pdf) agent to get high scores for the [CartPole](https://gym.openai.com/envs/CartPole-v0) environment. 

We make the agent train and test on OpenAI Gym toolkit's GUI interface provided through a distributed infrastructure (TCP API). More details can be found [here](https://github.com/zoq/gym_tcp_api).

A video of the trained agent can be seen in the end.

## Including necessary libraries and namespaces

In [1]:
#include <mlpack/xeus-cling.hpp>

In [2]:
#include <mlpack/core>
#include <mlpack/methods/ann/ffn.hpp>
#include <mlpack/methods/reinforcement_learning/q_learning.hpp>
#include <mlpack/methods/reinforcement_learning/q_networks/simple_dqn.hpp>
#include <mlpack/methods/reinforcement_learning/environment/env_type.hpp>
#include <mlpack/methods/reinforcement_learning/policy/greedy_policy.hpp>
#include <mlpack/methods/reinforcement_learning/training_config.hpp>

In [3]:
// Used to run the agent on gym's environment (provided externally) for testing.
#include <gym/environment.hpp>

In [4]:
// Used to generate and display a video of the trained agent.
#include "xwidgets/ximage.hpp"
#include "xwidgets/xvideo.hpp"
#include "xwidgets/xaudio.hpp"

In [5]:
using namespace mlpack;

In [6]:
using namespace mlpack::ann;

In [7]:
using namespace ens;

In [8]:
using namespace mlpack::rl;

## Initializing the agent

In [9]:
// Set up the state and action space.
DiscreteActionEnv::State::dimension = 4;
DiscreteActionEnv::Action::size = 2;

In [10]:
// Set up the network.
SimpleDQN<> model(DiscreteActionEnv::State::dimension, 128, 32,
                  DiscreteActionEnv::Action::size);

In [11]:
// Set up the policy and replay method.
GreedyPolicy<DiscreteActionEnv> policy(1.0, 1000, 0.1, 0.99);
RandomReplay<DiscreteActionEnv> replayMethod(32, 2000);

In [12]:
// Set up training configurations.
TrainingConfig config;
config.StepSize() = 0.001;
config.Discount() = 0.99;
config.TargetNetworkSyncInterval() = 100;
config.ExplorationSteps() = 100;
config.DoubleQLearning() = false;
config.StepLimit() = 200;

In [13]:
// Set up DQN agent.
QLearning<DiscreteActionEnv, decltype(model), AdamUpdate, decltype(policy)>
    agent(config, model, policy, replayMethod);

## Preparation for training the agent

In [14]:
// Set up the gym training environment.
gym::Environment env("gym.kurg.org", "4040", "CartPole-v0");

// Initializing training variables.
std::vector<double> returnList;
size_t episodes = 0;
bool converged = true;

// The number of episode returns to keep track of.
size_t consecutiveEpisodes = 50;

In [15]:
// Function to train the agent on mlpack's own implementation of the CartPole environment.
void Train(const size_t numSteps)
{
  agent.Deterministic() = false;
  std::cout << "Training for " << numSteps << " steps." << std::endl;
  while (agent.TotalSteps() < numSteps)
  {
    double episodeReturn = 0;
    env.reset();
    do
    {
      agent.State().Data() = env.observation;
      agent.SelectAction();
      arma::mat action = {double(agent.Action().action)};

      env.step(action);
      DiscreteActionEnv::State nextState;
      nextState.Data() = env.observation;

      replayMethod.Store(agent.State(), agent.Action(), env.reward, nextState,
          env.done, 0.99);
      episodeReturn += env.reward;
      agent.TotalSteps()++;
      if (agent.Deterministic() || agent.TotalSteps() < config.ExplorationSteps())
        continue;
      agent.TrainAgent();
    } while (!env.done);
    returnList.push_back(episodeReturn);
    episodes += 1;

    if (returnList.size() > consecutiveEpisodes)
      returnList.erase(returnList.begin());
        
    double averageReturn = std::accumulate(returnList.begin(),
                                           returnList.end(), 0.0) /
                           returnList.size();
    if(episodes % 1 == 0)
    {
      std::cout << "Avg return in last " << consecutiveEpisodes
          << " episodes: " << averageReturn
          << "\t Episode return: " << episodeReturn
          << "\t Total steps: " << agent.TotalSteps() << std::endl;
    }
  }
}

## Let the training begin

In [16]:
// Training the agent for a total of at least 2500 steps.
Train(2500)

Training for 2500 steps.
Avg return in last 50 episodes: 19	 Episode return: 19	 Total steps: 19
Avg return in last 50 episodes: 16.5	 Episode return: 14	 Total steps: 33
Avg return in last 50 episodes: 15	 Episode return: 12	 Total steps: 45
Avg return in last 50 episodes: 15	 Episode return: 15	 Total steps: 60
Avg return in last 50 episodes: 14.4	 Episode return: 12	 Total steps: 72
Avg return in last 50 episodes: 15.3333	 Episode return: 20	 Total steps: 92
Avg return in last 50 episodes: 14.8571	 Episode return: 12	 Total steps: 104
Avg return in last 50 episodes: 17.5	 Episode return: 36	 Total steps: 140
Avg return in last 50 episodes: 17.3333	 Episode return: 16	 Total steps: 156
Avg return in last 50 episodes: 17.6	 Episode return: 20	 Total steps: 176
Avg return in last 50 episodes: 20	 Episode return: 44	 Total steps: 220
Avg return in last 50 episodes: 19.1667	 Episode return: 10	 Total steps: 230
Avg return in last 50 episodes: 18.7692	 Episode return: 14	 Total steps: 244

## Testing the trained agent

In [17]:
agent.Deterministic() = true;

// Creating and setting up the gym environment for testing.
gym::Environment envTest("gym.kurg.org", "4040", "CartPole-v0");
envTest.monitor.start("./dummy/", true, true);

// Resets the environment.
envTest.reset();
envTest.render();

double totalReward = 0;
size_t totalSteps = 0;

// Testing the agent on gym's environment.
while (1)
{
  // State from the environment is passed to the agent's internal representation.
  agent.State().Data() = envTest.observation;

  // With the given state, the agent selects an action according to its defined policy.
  agent.SelectAction();

  // Action to take, decided by the policy.
  arma::mat action = {double(agent.Action().action)};

  envTest.step(action);
  totalReward += envTest.reward;
  totalSteps += 1;

  if (envTest.done)
  {
    std::cout << " Total steps: " << totalSteps << "\t Total reward: "
        << totalReward << std::endl;
    break;
  }

  // Uncomment the following lines to see the reward and action in each step.
  // std::cout << " Current step: " << totalSteps << "\t current reward: "
  //   << totalReward << "\t Action taken: " << action;
}

envTest.close();
std::string url = envTest.url();

auto video = xw::video_from_url(url).finalize();
video

 Total steps: 75	 Total reward: 75


A Jupyter widget

## A little more training...

In [18]:
// Training the same agent for a total of at least 5000 episodes.
Train(5000)

Training for 5000 steps.
Avg return in last 50 episodes: 36.56	 Episode return: 63	 Total steps: 2622
Avg return in last 50 episodes: 38.08	 Episode return: 90	 Total steps: 2712
Avg return in last 50 episodes: 38.98	 Episode return: 62	 Total steps: 2774
Avg return in last 50 episodes: 42.78	 Episode return: 200	 Total steps: 2974
Avg return in last 50 episodes: 46.22	 Episode return: 190	 Total steps: 3164
Avg return in last 50 episodes: 50	 Episode return: 200	 Total steps: 3364
Avg return in last 50 episodes: 52.36	 Episode return: 127	 Total steps: 3491
Avg return in last 50 episodes: 56	 Episode return: 195	 Total steps: 3686
Avg return in last 50 episodes: 58.86	 Episode return: 152	 Total steps: 3838
Avg return in last 50 episodes: 62.5	 Episode return: 193	 Total steps: 4031
Avg return in last 50 episodes: 66.28	 Episode return: 200	 Total steps: 4231
Avg return in last 50 episodes: 69.98	 Episode return: 200	 Total steps: 4431
Avg return in last 50 episodes: 73.32	 Episode re

# Final agent testing!

In [19]:
agent.Deterministic() = true;

// Creating and setting up the gym environment for testing.
gym::Environment envTest("gym.kurg.org", "4040", "CartPole-v0");
envTest.monitor.start("./dummy/", true, true);

// Resets the environment.
envTest.reset();
envTest.render();

double totalReward = 0;
size_t totalSteps = 0;

// Testing the agent on gym's environment.
while (1)
{
  // State from the environment is passed to the agent's internal representation.
  agent.State().Data() = envTest.observation;

  // With the given state, the agent selects an action according to its defined policy.
  agent.SelectAction();

  // Action to take, decided by the policy.
  arma::mat action = {double(agent.Action().action)};

  envTest.step(action);
  totalReward += envTest.reward;
  totalSteps += 1;

  if (envTest.done)
  {
    std::cout << " Total steps: " << totalSteps << "\t Total reward: "
        << totalReward << std::endl;
    break;
  }

  // Uncomment the following lines to see the reward and action in each step.
  // std::cout << " Current step: " << totalSteps << "\t current reward: "
  //   << totalReward << "\t Action taken: " << action;
}

envTest.close();
std::string url = envTest.url();

auto video = xw::video_from_url(url).finalize();
video

 Total steps: 144	 Total reward: 144


A Jupyter widget